In [1]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.0f}'.format

In [2]:
testing_df = pd.read_csv('https://healthdata.gov/api/views/j8mb-icvb/rows.csv?accessType=DOWNLOAD')
testing_df.head(5)

,state,state_name,state_fips,fema_region,overall_outcome,date,new_results_reported,total_results_reported,geocoded_state
0,AL,Alabama,1,Region 4,Negative,2020/03/01,96,96,nan
1,AL,Alabama,1,Region 4,Positive,2020/03/01,16,16,nan
2,AL,Alabama,1,Region 4,Negative,2020/03/02,72,168,nan
3,AL,Alabama,1,Region 4,Positive,2020/03/02,6,22,nan
4,AL,Alabama,1,Region 4,Negative,2020/03/03,94,262,nan


In [3]:
positive = testing_df[(testing_df['overall_outcome'] == 'Positive')]
positive.head()

,state,state_name,state_fips,fema_region,overall_outcome,date,new_results_reported,total_results_reported,geocoded_state
1,AL,Alabama,1,Region 4,Positive,2020/03/01,16,16,nan
3,AL,Alabama,1,Region 4,Positive,2020/03/02,6,22,nan
5,AL,Alabama,1,Region 4,Positive,2020/03/03,9,31,nan
7,AL,Alabama,1,Region 4,Positive,2020/03/04,2,33,nan
9,AL,Alabama,1,Region 4,Positive,2020/03/05,6,39,nan


In [4]:
negative = testing_df[(testing_df['overall_outcome'] == 'Negative')]
negative.head()

,state,state_name,state_fips,fema_region,overall_outcome,date,new_results_reported,total_results_reported,geocoded_state
0,AL,Alabama,1,Region 4,Negative,2020/03/01,96,96,nan
2,AL,Alabama,1,Region 4,Negative,2020/03/02,72,168,nan
4,AL,Alabama,1,Region 4,Negative,2020/03/03,94,262,nan
6,AL,Alabama,1,Region 4,Negative,2020/03/04,0,262,nan
8,AL,Alabama,1,Region 4,Negative,2020/03/05,61,323,nan


In [5]:
results_df = pd.merge(positive, negative, left_on=['state', 'state_name', 'state_fips', 'fema_region', 'date'], right_on=['state', 'state_name', 'state_fips', 'fema_region', 'date'])
results_df.head()

,state,state_name,state_fips,fema_region,overall_outcome_x,date,new_results_reported_x,total_results_reported_x,geocoded_state_x,overall_outcome_y,new_results_reported_y,total_results_reported_y,geocoded_state_y
0,AL,Alabama,1,Region 4,Positive,2020/03/01,16,16,nan,Negative,96,96,nan
1,AL,Alabama,1,Region 4,Positive,2020/03/02,6,22,nan,Negative,72,168,nan
2,AL,Alabama,1,Region 4,Positive,2020/03/03,9,31,nan,Negative,94,262,nan
3,AL,Alabama,1,Region 4,Positive,2020/03/04,2,33,nan,Negative,0,262,nan
4,AL,Alabama,1,Region 4,Positive,2020/03/05,6,39,nan,Negative,61,323,nan


In [6]:
results2 = results_df.rename({'new_results_reported_x': 'new_positive', 'new_results_reported_y': 'new_negative', 'total_results_reported_x': 'total_positive', 'total_results_reported_y': 'total_negative'}, axis=1)
results = results2.drop(columns=['overall_outcome_x', 'overall_outcome_y', 'geocoded_state_x', 'geocoded_state_y'])
results.head()

,state,state_name,state_fips,fema_region,date,new_positive,total_positive,new_negative,total_negative
0,AL,Alabama,1,Region 4,2020/03/01,16,16,96,96
1,AL,Alabama,1,Region 4,2020/03/02,6,22,72,168
2,AL,Alabama,1,Region 4,2020/03/03,9,31,94,262
3,AL,Alabama,1,Region 4,2020/03/04,2,33,0,262
4,AL,Alabama,1,Region 4,2020/03/05,6,39,61,323


In [7]:
results['daily_tests'] = results['new_positive'] + results['new_negative']
results['cumulative'] = results['total_positive'] + results['total_negative']
results

,state,state_name,state_fips,fema_region,date,new_positive,total_positive,new_negative,total_negative,daily_tests,cumulative
0,AL,Alabama,1,Region 4,2020/03/01,16,16,96,96,112,112
1,AL,Alabama,1,Region 4,2020/03/02,6,22,72,168,78,190
2,AL,Alabama,1,Region 4,2020/03/03,9,31,94,262,103,293
3,AL,Alabama,1,Region 4,2020/03/04,2,33,0,262,2,295
4,AL,Alabama,1,Region 4,2020/03/05,6,39,61,323,67,362
...,...,...,...,...,...,...,...,...,...,...,...
23654,WY,Wyoming,56,Region 8,2021/05/10,47,51199,530,717235,577,768434
23655,WY,Wyoming,56,Region 8,2021/05/11,89,51288,950,718185,1039,769473
23656,WY,Wyoming,56,Region 8,2021/05/12,54,51342,1771,719956,1825,771298
23657,WY,Wyoming,56,Region 8,2021/05/13,56,51398,1774,721730,1830,773128


In [8]:
results.to_csv('covid_tests.csv', index=False)

In [9]:
cal_tests = results[results['state_name'] == 'California']
cal_tests.to_csv('ca_covid_tests.csv', index=False)